<a href="https://colab.research.google.com/github/beniguha/GenAI/blob/main/Agentic-AI-Experiments/Tool15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from rich.console import Console
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
from google.colab import userdata
load_dotenv(override=True)

False

In [10]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [11]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
openai = OpenAI()

In [12]:
# Some lists!

todos = []
completed = []

In [13]:
def get_todo_report() -> str:
    result = ""
    for index, todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index + 1}: {todo}\n"
    show(result)
    return result

In [14]:
get_todo_report()

''

In [15]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [16]:
def mark_complete(index: int, completion_notes: str) -> str:
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [26]:
todos, completed = [], []

create_todos(["Buy groceries", "Finish extra lab", "Eat banana"])

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: Buy groceries\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [27]:
mark_complete(1, "bought")

bought

Todo #1: Buy groceries
Todo #2: Finish extra lab
Todo #3: Eat banana

'Todo #1: [green][strike]Buy groceries[/strike][/green]\nTodo #2: Finish extra lab\nTodo #3: Eat banana\n'

In [19]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [20]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [21]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [22]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [23]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools, reasoning_effort="none")
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)

In [ ]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [24]:
system_message = """
You are given a problem to solve, by using your todo tools to plan a list of steps, then carrying out each step in turn.
Now use the todo list tools, create a plan, carry out the steps, and reply with the solution.
If any quantity isn't provided in the question, then include a step to come up with a reasonable estimate.
Provide your solution in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
A train leaves Boston at 2:00 pm traveling 60 mph.
Another train leaves New York at 3:00 pm traveling 80 mph toward Boston.
When do they meet?
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [25]:
todos, completed = [], []
loop(messages)

Todo #1: Interpret the problem setup and determine initial separation and relative motion model.
Todo #2: Estimate/assume Boston–New York distance since not provided.
Todo #3: Compute each train’s distance traveled as functions of time and solve for meeting time.
Todo #4: Convert solution to clock time and present clearly.

Model: two trains on the same line between Boston and New York, moving toward each other. Let t be hours after 2:00
pm. Boston train travels 60t miles. New York train starts at 3:00 pm, i.e., at t=1, and then travels 80(t-1) miles 
for t≥1. They meet when the sum of distances equals the initial separation.

Todo #1: Interpret the problem setup and determine initial separation and relative motion model.
Todo #2: Estimate/assume Boston–New York distance since not provided.
Todo #3: Compute each train’s distance traveled as functions of time and solve for meeting time.
Todo #4: Convert solution to clock time and present clearly.

Distance not given; use a reasonable rail/highway corridor estimate. A common approximation for BostonNew York is
about 215 miles (NYCBoston is often quoted ~215220 mi). Proceed with 215 mi as the separation.

Todo #1: Interpret the problem setup and determine initial separation and relative motion model.
Todo #2: Estimate/assume Boston–New York distance since not provided.
Todo #3: Compute each train’s distance traveled as functions of time and solve for meeting time.
Todo #4: Convert solution to clock time and present clearly.

Solve for t≥1: 60t + 80(t-1) = 215.

Compute: 60t + 80t - 80 = 215 → 140t = 295 → t = 295/140 ≈ 2.1071 hours after 2:00 pm.

Todo #1: Interpret the problem setup and determine initial separation and relative motion model.
Todo #2: Estimate/assume Boston–New York distance since not provided.
Todo #3: Compute each train’s distance traveled as functions of time and solve for meeting time.
Todo #4: Convert solution to clock time and present clearly.

Convert t≈2.1071 h to clock time: 2 h + 0.1071 h. 0.1071 h × 60 ≈ 6.43 min ≈ 6 min 26 s. Add to 2:00 pm → about 
4:06 pm (and 26 s).

Todo #1: Interpret the problem setup and determine initial separation and relative motion model.
Todo #2: Estimate/assume Boston–New York distance since not provided.
Todo #3: Compute each train’s distance traveled as functions of time and solve for meeting time.
Todo #4: Convert solution to clock time and present clearly.

Assuming Boston–New York is about 215 miles apart (distance not provided):

Let t = hours after 2:00 pm.

- Boston train distance: 60t
- New York train leaves at 3:00 pm ⇒ starts at t = 1, distance: 80(t − 1) for t ≥ 1

They meet when:
60t + 80(t − 1) = 215  
140t − 80 = 215  
140t = 295  
t = 295/140 ≈ 2.107 hours after 2:00 pm

2.107 hours = 2 hours + 0.107×60 ≈ 2 hours + 6.43 minutes.

They meet at about 4:06 pm (approximately 4:06:26 pm).